In [1]:
from queue import PriorityQueue, LifoQueue
import numpy as np
import random as rnd
import copy

In [2]:
class Cell:     
    OPEN = 0
    BLOCKED = -1
    ON_FIRE = 2
    NO_PREV = (-1, -1)

    def __init__(self, coords, prev = None, f = 0, g = 0, h = -1):
        self.coords = coords
        self.prev = prev
        self.f = f
        self.g = g
        self.h = h
        
    def __lt__(self, other):   
        if self.f == other.f:
            if self.h == other.h:
                if self.g == other.g:
                    return self.coords > other.coords
                return self.g < other.g
            return 
        return self.f < other.f
    
    def __eq__(self, other):
        return self.coords == other.coords
    
    def __repr__(self):
        return str(self.coords) + " :: parent=" + ("NONE" if self.prev is None else str(self.prev.coords))

In [3]:
class Maze:
    def __init__(self, dim, p, q = 0):
        self.board =  np.zeros([dim, dim], dtype=int)
        self.dim = dim
        self.currentfire = []
        self.q = q
        self.p = p
        self.board_prob = np.zeros(shape=(dim**2,dim**2))
        self.fires = np.array([])
        obstaclesList = []
        for i in range(0, dim): #range() generates a list from 0 to dim-1 which can take up a lot of memory
            for j in range(0, dim):             
                if(rnd.random() < p):
                    self.board[i, j] = Cell.BLOCKED
                    obstaclesList.append((i, j))
        self.obstacles = np.array(obstaclesList)
        self.board[0, 0] = Cell.OPEN
        self.board[dim - 1, dim - 1] = Cell.OPEN

    def isSolvable(self):
        return pathExists(self)
    
    def startFire(self):
        i = rnd.randint(0,self.dim-1)
        j = rnd.randint(0,self.dim-1)
        while(self.board[i,j] == -1 or (i==0 and j == 0) or (i == self.dim-1 and j == self.dim-1)):
            i = rnd.randint(0,self.dim-1)
            j = rnd.randint(0,self.dim-1)
        self.board[i,j] = Cell.ON_FIRE
        self.currentfire.append([i,j,None])
        self.fires = np.append(self.fires, [tupleToIndex(i, j, self.dim)])
    
    def iterateFire(self):
        visited = [] #neighbors already visited in current iteration
        nowonfire = [] #need to append all new fires at the end 
        for f in self.currentfire:
            if(f[2] != 0): #check to see if all neighbors are already on fire. 
                neighbors = findNeighboringFlammableCoords(f[0:2],self) #calculate valid neighbors of f, the cell that is currently on fire
                ofnbor = 0 #counter to find out how many neighbors are on fire
                for n in neighbors: #loop through the neighbors of f
                    nr,nc = n 
                    if (self.board[nr,nc] == Cell.ON_FIRE): 
                        ofnbor = ofnbor + 1 
                    else:
                        if(n[0:2] not in visited): #only roll once for each cell not on fire
                            if(self.board[nr,nc] != Cell.BLOCKED):
                                surroundings = findNeighboringFlammableCoords(n[0:2],self) #see what the neighbors are for the neighbor of f
                                k = 0 #k will be guaranteed to be at least 1
                                for nn in surroundings:
                                    nnr,nnc = nn
                                    if(self.board[nnr,nnc] == Cell.ON_FIRE):
                                        k = k + 1 #counter
                                if(rnd.random() <= (1-((1-self.q)**k))): #roll
                                    self.board[nr,nc] = Cell.ON_FIRE 
                                    nowonfire.append([nr,nc,None])
                                    ofnbor = ofnbor + 1
                                else:
                                    visited.append([nr,nc])
                if(ofnbor == len(neighbors)):
                        f[2] = 0
        if(nowonfire):
            for x in nowonfire:
                self.currentfire.append(x)
                self.fires = np.append(self.fires, tupleToIndex(x[0], x[1], self.dim))
            return True
        return False

In [4]:
def pathExists(maze, fromCoords = (0,0), toCoords = None):
    if toCoords is None:
        toCoords = (maze.dim - 1, maze.dim - 1)
    toRow, toCol = toCoords
    visited = set()
    fringe = LifoQueue()
    fringe.put(fromCoords)
    while not fringe.empty():
        currentCoords = fringe.get()
        currentRow, currentCol = currentCoords
        if (currentRow == toRow and currentCol == toCol):
            return True
        else:
            neighbors = findNeighboringOpenCoords(currentCoords, maze)
            for neighbor in neighbors:
                if neighbor in visited:
                    continue
                fringe.put(neighbor)
            visited.add(currentCoords)
    return False

In [5]:
def findNeighboringOpenCoords(coords, maze):
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] != Cell.OPEN):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

def findNeighboringFlammableCoords(coords,maze): 
    cellRow, cellCol = coords
    potentialNeighbors = [(cellRow + 1, cellCol), (cellRow - 1, cellCol), (cellRow, cellCol - 1), (cellRow, cellCol + 1)]
    neighbors = []
    for potentialNeighbor in potentialNeighbors:
        row, col = potentialNeighbor
        if (row >= maze.dim or row < 0 or col >= maze.dim or col < 0 or maze.board[row,col] == Cell.BLOCKED):
            continue
        neighbors.append(potentialNeighbor)
    return neighbors

In [6]:
def uniformCost(cell, maze, visited):
    return 0

def shortestPathSearch(maze, startCoords = (0, 0), goalCoords = None, heuristicFunction = uniformCost, findNeighborsFunction = findNeighboringOpenCoords):
    if goalCoords is None:
        goalCoords = (maze.dim - 1, maze.dim - 1)
    startCell = Cell(startCoords)
    visited = set()
    fringe = PriorityQueue()
    fringe.put(startCell)
    heuristicVisited = {}
    expandedCells = 0
    while not fringe.empty():
        currentCell = fringe.get()
        if (currentCell.coords) in visited:
            continue
        if (currentCell.coords == goalCoords):
            shortestPath = []
            while (currentCell is not None):
                shortestPath.append(currentCell.coords)
                currentCell = currentCell.prev
            shortestPath.reverse()
            return (shortestPath, expandedCells)
        else:
            neighborsCoordsList = findNeighborsFunction(coords = currentCell.coords, maze = maze)
            for neighborCoords in neighborsCoordsList:
                if neighborCoords in visited:
                    continue
                neighbor = Cell(neighborCoords, prev = currentCell)
                neighbor.g = currentCell.g + 1
                neighbor.h = heuristicFunction(cell = neighbor, maze = maze, visited = heuristicVisited)
                neighbor.f = neighbor.g + neighbor.h
                fringe.put(neighbor)
            visited.add(currentCell.coords)
            expandedCells += 1
    return None

In [7]:
def simulationHeuristic(cell, maze, visited):
    #The weight is equal to the probability that a cell catches fire at a specific timestep
    #cell.g stores the smallest possible timestep that that specific cell can be reached
    weight = maze.board_prob[tupleToIndex(cell.coords[0], cell.coords[1], maze.dim)][int(cell.g)] 
    return weight

In [8]:
import copy
def run_strat4(maze):
    num_sims = 30
    for x in range(num_sims): 
        copiedMaze = copy.deepcopy(maze) #deepcopies the inputted maze to reset fire upon every simulation
        oldFires = np.array([])
        for y in range(maze.dim**2): #the upper bound on the total number of timesteps is dim squared
            oldFires = copy.deepcopy(copiedMaze.fires) #deepcopies the current mazes's fires before advancing the fire
            if(copiedMaze.iterateFire()):
                newFires = np.setdiff1d(copiedMaze.fires, oldFires) #stores the newly created fires
                for f in newFires:
                    maze.board_prob[int(f)][y] += 1.0 #increments the specific cell, timestep count
    for index in range(maze.dim**2):
        maze.board_prob[index] = np.divide(maze.board_prob[index], num_sims) #divides each ignitition count by the number of simulations
    
    agent = (0,0)
    while True:
        result = shortestPathSearch(maze, startCoords = agent, heuristicFunction = simulationHeuristic) #runs the shortest path algorithm using the probability heuristic
        if result is not None:
            shortestPath = result[0] #deconstructs the result if it exists
        if shortestPath != None:
            agent = shortestPath[1] #moves the agent if there is a shortest path
        if agent == (maze.dim-1, maze.dim-1):
            return True #returns true if the agent has reached the goal
        maze.iterateFire()
        if maze.board[agent] == 2:
            return False #returns false if the agent gets burned by the fire

In [9]:
def tupleToIndex(r, c, dim):
    return dim * r + c #conv
def indexToTuple(i, dim):
    return (int(i / dim), i % dim)

In [10]:
m = Maze(10, 0.3, 0.3)
while(m.isSolvable() == False):
    m = Maze(10, 0.3, 0.3)
m.startFire()
print(run_strat4(m))

True
